In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor, XGBClassifier
import warnings

In [88]:
df = pd.read_csv(r'C:\Users\rksai\OneDrive\Desktop\Telco Customer Churn\data\WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [89]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [90]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Converting total charges into numeric

In [91]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

In [92]:
df['TotalCharges'].isnull().sum()

0

Droping the unnecessary column

In [93]:
df.drop("customerID", axis=1, inplace=True)

In [94]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [96]:
cat_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'InternetService', 'Contract', 'PaymentMethod', 'MultipleLines']

num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [97]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [98]:
cat_tranformer = OneHotEncoder()
num_transformer = StandardScaler()

prepreocessor = ColumnTransformer([

    ("OneHotEncoder", cat_tranformer, cat_cols),
    ("StandardScaler", num_transformer, num_cols)
]
)


In [99]:
df['Churn']=df['Churn'].apply(lambda x:1 if x=='Yes' else 0)

In [100]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


Splitting dataset

In [101]:
x = df.drop(columns=['Churn'],axis=1)
y = df['Churn']

In [102]:
x

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60


In [103]:
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [104]:
x = prepreocessor.fit_transform(x)

In [105]:
x

array([[ 0.        ,  1.        ,  1.        , ..., -1.27744458,
        -1.16032292, -0.99424194],
       [ 1.        ,  0.        ,  1.        , ...,  0.06632742,
        -0.25962894, -0.17324413],
       [ 1.        ,  0.        ,  1.        , ..., -1.23672422,
        -0.36266036, -0.95967407],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -0.87024095,
        -1.1686319 , -0.85446945],
       [ 0.        ,  1.        ,  1.        , ..., -1.15528349,
         0.32033821, -0.87206242],
       [ 1.        ,  0.        ,  1.        , ...,  1.36937906,
         1.35896134,  2.01428802]])

Splitting into train and test data

In [106]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
print(f"the shape of x_train is : {x_train.shape}")
print(f'the shape of x_test is : {x_test.shape}')
print(f'the shape of y_tain is : {y_train.shape}')
print(f'the shape of y_test is {y_test.shape}')

the shape of x_train is : (5282, 24)
the shape of x_test is : (1761, 24)
the shape of y_tain is : (5282,)
the shape of y_test is (1761,)


Training the model

In [107]:
logreg = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()
dt = DecisionTreeClassifier()

In [108]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

LOGISTIC REGRESSION

In [109]:
model1 = logreg.fit(x_train, y_train)
y_pred = model1.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1282
           1       0.68      0.56      0.61       479

    accuracy                           0.81      1761
   macro avg       0.76      0.73      0.74      1761
weighted avg       0.80      0.81      0.80      1761

ROC-AUC: 0.7306075124007048
Confusion Matrix:
 [[1156  126]
 [ 211  268]]


RANDOM FOREST

In [110]:
model2 = rf.fit(x_train, y_train)
y_pred = model2.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1282
           1       0.64      0.50      0.56       479

    accuracy                           0.79      1761
   macro avg       0.73      0.70      0.71      1761
weighted avg       0.78      0.79      0.78      1761

ROC-AUC: 0.6977436091180599
Confusion Matrix:
 [[1144  138]
 [ 238  241]]


XGBCLASSIFIER

In [111]:
model3 = xgb.fit(x_train, y_train)
y_pred = model3.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1282
           1       0.62      0.49      0.55       479

    accuracy                           0.78      1761
   macro avg       0.72      0.69      0.70      1761
weighted avg       0.77      0.78      0.77      1761

ROC-AUC: 0.6895304830982384
Confusion Matrix:
 [[1139  143]
 [ 244  235]]


DECISION TREE

In [121]:
model4 = dt.fit(x_train, y_train)
y_pred = model4.predict(x_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      1282
           1       0.52      0.52      0.52       479

    accuracy                           0.74      1761
   macro avg       0.67      0.67      0.67      1761
weighted avg       0.74      0.74      0.74      1761

ROC-AUC: 0.6683890320122199
Confusion Matrix:
 [[1050  232]
 [ 231  248]]


Hyperparameter Tuning

In [113]:
models = {
    
    "logreg": LogisticRegression(),
    "rf": RandomForestClassifier(),
    "xgb": XGBClassifier(),
    "dt": DecisionTreeClassifier()


}

params = {
    
    "logreg": {
        
        "C": [0.01, 0.1, 1, 10],
        "penalty": ["l2"], 
        "solver": ["lbfgs", "liblinear"],
        "class_weight": ["balanced", None]
},

"rf": {

    "n_estimators": [100, 200, 300],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "class_weight": ["balanced", "balanced_subsample", None]
},

"xgb": {

    "n_estimators": [200, 300, 500],
    "max_depth": [3, 6, 10],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.8, 1],
    "colsample_bytree": [0.8, 1]


},

"dt": {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [None, 5, 10, 20, 30],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 5, 10],
    "max_features": [None, "sqrt", "log2"]

}

}



for name, model in models.items():
    print(f"🔎 Running GridSearchCV for {name}...")
    gs = GridSearchCV(model, params[name], cv=3, scoring="roc_auc", n_jobs=-1, verbose=1)
    gs.fit(x_train, y_train)

🔎 Running GridSearchCV for logreg...
Fitting 3 folds for each of 16 candidates, totalling 48 fits
🔎 Running GridSearchCV for rf...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
🔎 Running GridSearchCV for xgb...
Fitting 3 folds for each of 108 candidates, totalling 324 fits
🔎 Running GridSearchCV for dt...
Fitting 3 folds for each of 720 candidates, totalling 2160 fits


Best Model

In [122]:
best_model = gs.best_estimator_
best_model

DecisionTreeClassifier(criterion='log_loss', max_depth=5)

In [123]:
gs.best_params_

{'criterion': 'log_loss',
 'max_depth': 5,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}